<a href="https://colab.research.google.com/github/grace3999/USV_Python/blob/colab/6_xr_Annotations_deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install netcdf4

    100% |████████████████████████████████| 3.9MB 8.2MB/s 
    100% |████████████████████████████████| 307kB 32.2MB/s 


In [0]:
import numpy as np
import pandas as pd
from os.path import join
#import datetime
#from scipy import signal
#from scipy import stats
from scipy import interpolate
import xarray as xr

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve


#visualizing results
%matplotlib inline
import matplotlib.pyplot as plt
#import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [0]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
#mount google drive containings required files: 1) csv of annotation features, 2) netcdf files
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
#use shell commands to find file path for csv of annotation features
features_path = '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/data_frames/annot_features_full.csv'
annot_features_data = pd.read_csv(features_path)
annot_features_full = pd.DataFrame(data = annot_features_data)
print(annot_features_full.shape)
#ensure no nans
print(annot_features_full.loc[pd.isna(annot_features_full['spec_pur'])].index)
annot_features_full.head()

(959, 13)
Int64Index([], dtype='int64')


,Unnamed: 0,animal_number,session,time_stamp,Annotation,power_sum,spec_pur,spec_cent,spec_spread,spec_skew,spec_kurt,spec_slope,spec_roll
0,0,527,cagepair,18922.5,high slug,44921.600,0.335974,40617.050954,117729.192000,6.831866,48.160918,0.000787,38586.198407
1,1,527,cagepair,24750.0,low slug,53212.637,0.295599,27042.548508,3827.200195,3.556462,20.402396,-0.000227,25690.421083
2,2,527,cagepair,56002.5,bbc,94253.766,0.189555,30090.095178,68103.292456,3.982554,18.393372,-0.002314,28585.590419
3,3,527,cagepair,174352.5,bbc,51008.836,0.306177,34324.170621,52563.703613,3.336529,10.448164,-0.001250,32607.962090
4,4,527,cagepair,342877.5,bbc,164846.110,0.144833,33090.921412,22861.514782,1.852232,3.185318,-0.001784,31436.375341


In [7]:
#create array of class labels, add to df
annot_labels = annot_features_full['Annotation']
annot_labels = annot_labels.replace(regex={r'low slug': 1, 'high slug': 1, 'low multi': 1, 'high multi': 1, 'bbc': 1, 'rand_noise': 0})
annot_features_full['class'] = annot_labels
annot_features_full.head()

,Unnamed: 0,animal_number,session,time_stamp,Annotation,power_sum,spec_pur,spec_cent,spec_spread,spec_skew,spec_kurt,spec_slope,spec_roll,class
0,0,527,cagepair,18922.5,high slug,44921.600,0.335974,40617.050954,117729.192000,6.831866,48.160918,0.000787,38586.198407,1
1,1,527,cagepair,24750.0,low slug,53212.637,0.295599,27042.548508,3827.200195,3.556462,20.402396,-0.000227,25690.421083,1
2,2,527,cagepair,56002.5,bbc,94253.766,0.189555,30090.095178,68103.292456,3.982554,18.393372,-0.002314,28585.590419,1
3,3,527,cagepair,174352.5,bbc,51008.836,0.306177,34324.170621,52563.703613,3.336529,10.448164,-0.001250,32607.962090,1
4,4,527,cagepair,342877.5,bbc,164846.110,0.144833,33090.921412,22861.514782,1.852232,3.185318,-0.001784,31436.375341,1


In [8]:
#separate and save for later testing dataset
#stratify splitting by annotation types
train_orig, test_orig = train_test_split(annot_features_full, test_size=0.3, random_state=1, stratify = annot_features_full.Annotation)
print(train_orig.shape)
print(test_orig.shape)
print(train_orig['Annotation'].value_counts())

(671, 14)
(288, 14)
rand_noise    560
low slug       42
bbc            31
high slug      19
low multi      16
high multi      3
Name: Annotation, dtype: int64


In [16]:
#separate and save for later testing dataset
train_validate, test_validate = train_test_split(train_orig, test_size=0.15, random_state=1, stratify = train_orig.Annotation)
print(train_validate.shape)
print(test_validate.shape)
print(train_validate['Annotation'].value_counts())
print(test_validate['Annotation'].value_counts())

(570, 14)
(101, 14)
rand_noise    476
low slug       36
bbc            26
high slug      16
low multi      14
high multi      2
Name: Annotation, dtype: int64
rand_noise    84
low slug       6
bbc            5
high slug      3
low multi      2
high multi     1
Name: Annotation, dtype: int64


In [10]:
#upsample minority class (upsample to an equal number of noise samples)
train_validate_maj = train_validate[train_validate['Annotation'] == 'rand_noise']
train_validate_min = train_validate[train_validate['Annotation'] != 'rand_noise']

train_validate_min_upsamples = resample(train_validate_min, replace=True, n_samples = len(train_validate_maj), random_state = 123)

train_validate_upsamples = pd.concat([train_validate_min_upsamples, train_validate_maj])

print(train_validate_upsamples.shape)
print(train_validate_upsamples['Annotation'].value_counts())

train_validate_upsamples.head()

(952, 14)
rand_noise    476
low slug      170
bbc           125
high slug      88
low multi      80
high multi     13
Name: Annotation, dtype: int64


,Unnamed: 0,animal_number,session,time_stamp,Annotation,power_sum,spec_pur,spec_cent,spec_spread,spec_skew,spec_kurt,spec_slope,spec_roll,class
124,124,556,cagepair,41512.5,bbc,283922.600,0.105281,35128.433948,27613.082970,1.746885,2.369667,-0.001916,33372.012250,1
49,49,535,CPApair,201847.5,low slug,114278.914,0.151498,16245.921214,18541.400530,5.060017,27.840860,-0.001240,15433.625153,1
131,131,556,cagepair,169807.5,high slug,47409.770,0.329336,30112.858088,13748.241415,6.074303,42.912236,-0.000247,28607.215183,1
137,137,556,cagepair,459967.5,high slug,88313.625,0.185218,34136.696251,175651.225991,6.314609,40.124449,-0.001435,32429.861439,1
147,147,557,cagepair,79177.5,bbc,121944.500,0.143545,31621.628971,96620.582439,3.463390,12.100880,-0.002297,30040.547522,1


In [0]:
def square_rgb(xr_slice):
    #takes in slice from netcdf file and reshapes to be compatible with keras pretrained model
    times = data['times'].values
    freqs = data['freq'].values
    
    #make square for Keras - need to interpolate data, first create function and then apply
    x = np.arange(0, len(times))
    y = np.arange(0, len(freqs))
    f = interpolate.interp2d(x, y, xr_slice.values)
    xnew = np.linspace(0, len(times), num=128)
    ynew = np.linspace(0, len(freqs), num=128)
    znew = f(xnew, ynew)

    X = np.log(znew)
    
    #normalize and change to rgba (remove a)
    X_colored = plt.cm.viridis((X - X.mean())/(X.max() - X.min()))
    X_colored = X_colored[:,:,:3]
    
    return X_colored

In [0]:
data = xr.open_dataset('/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcdf_files/' + str(535) + '_xr_Dataset.nc')

In [15]:
#create keras compatible image for each annotation in train data set, save as np array
rgbs = []
i = 1
for index, row in train_validate_upsamples.iterrows():
    data = xr.open_dataset('/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcdf_files/' + str(row['animal_number']) + '_xr_Dataset.nc')
    
    xr_slice = data['__xarray_dataarray_variable__'].sel(slices=row['time_stamp'])
    
    rgb = square_rgb(xr_slice)
    
    rgbs.append(rgb)
    
X = np.stack(rgbs)
print(X.shape)

(952, 128, 128, 3)


In [17]:
y = train_validate_upsamples['class']
y.shape

(952,)

In [18]:
#create keras compatible image for each annotation in test data set, save as np array
rgbs_test = []
i = 1
for index, row in test_validate.iterrows():
    data = xr.open_dataset('/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcdf_files/' + str(row['animal_number']) + '_xr_Dataset.nc')
    
    xr_slice = data['__xarray_dataarray_variable__'].sel(slices=row['time_stamp'])
    
    rgb = square_rgb(xr_slice)

    rgbs_test.append(rgb)
    
X_test = np.stack(rgbs_test)
print(X_test.shape)

(101, 128, 128, 3)


In [19]:
y_test = test_validate['class']
y_test.shape

(101,)

In [20]:
import keras
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

from keras.layers import Dropout, Activation
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, TensorBoard, Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

Using TensorFlow backend.


In [21]:
base_model=MobileNet(input_shape = (128,128,3), include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

Instructions for updating:
Colocations handled automatically by placer.
17227776/17225924 [==============================] - 0s 0us/step


In [0]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
#x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
#x=Dense(1024,activation='relu')(x) #dense layer 2
#x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(1,activation='sigmoid', name='features')(x) #final layer with softmax activation

In [0]:
model=Model(inputs=base_model.input,outputs=preds)

In [0]:
for layer in model.layers[:-5]:
    layer.trainable=False
for layer in model.layers[-5:]:
    layer.trainable=True

In [0]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=[f1])

In [0]:
train_datagen=ImageDataGenerator(featurewise_center=True,
    featurewise_std_normalization=True, width_shift_range=0.2, preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow(X, y, batch_size=32,shuffle=True)

In [0]:
test_datagen = ImageDataGenerator()

validation_generator = test_datagen.flow(X_test, y_test, batch_size=32,shuffle=True)

In [0]:
log_path = '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/TF_logs'

In [0]:
train_validate_upsamples.to_csv('train_validate_upsamples.csv')

In [32]:
ls

gdrive/  sample_data/  train_validate_upsamples.csv


In [29]:
# Upload the file to Drive. See:
#
# https://developers.google.com/drive/v3/reference/files/create
# https://developers.google.com/drive/v3/web/manage-uploads
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'TF_log_run_a',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

FileNotFoundError: ignored

In [0]:
log_dir='/content/gdrive/Team Drives/USV_eScience_Incubator/Data/TF_logs/run_a'

with open(join(log_dir, 'metadata.tsv'), 'w') as f:
    f.write('Index\tLabel\n')
    for index,label in enumerate(y_test):
        f.write('%d\t%d\n' % (index,label))

tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1, batch_size=32, write_graph=True, write_grads=False, write_images=True, embeddings_freq=1, embeddings_layer_names=['features'], embeddings_metadata=join(log_dir, 'metadata.tsv'), embeddings_data=X_test, update_freq='epoch')


callbacks_list = [
    EarlyStopping(monitor = 'val_acc', patience = 6, verbose = 1),
    ReduceLROnPlateau(monitor = 'val_acc', factor = 0.1, patience = 3, verbose = 1), tensorboard]

In [0]:
step_size_train=train_generator.n//train_generator.batch_size

model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train,
                   epochs=9,
                   validation_data = [X_test, y_test],
                callbacks = callbacks_list)

#model.fit_generator(train_generator, steps_per_epoch=num_of_train_samples // batch_size, epochs=epochs, validation_data=validation_generator, validation_steps=num_of_test_samples // batch_size)

Instructions for updating:
Use tf.cast instead.
Epoch 1/9
29/29 [==============================] - 36s 1s/step - loss: 0.4628 - f1: 0.8182 - val_loss: 0.6539 - val_f1: 0.0000e+00
Epoch 2/9
29/29 [==============================] - 34s 1s/step - loss: 0.2647 - f1: 0.8999 - val_loss: 1.4092 - val_f1: 0.0000e+00
Epoch 3/9
29/29 [==============================] - 34s 1s/step - loss: 0.1519 - f1: 0.9369 - val_loss: 0.6482 - val_f1: 0.0000e+00


In [0]:
#command for starting tensorboard from terminal

#python -m tensorboard.main --logdir="C:/Users/Schindler/Documents/ProgrammingFun/USV_python/TF_logs/run_a"

In [0]:
#Confution Matrix and Classification Report
Y_pred = model.predict_generator(validation_generator)
y_pred = np.argmax(Y_pred, axis=1)

In [0]:
print(Y_pred.shape)
print(y_test.shape)

In [0]:
precision, recall, thresholds = precision_recall_curve(y_test, Y_pred)
plt.plot(recall, precision)

plt.xlim([0, 1])
plt.ylim([0, 1.05])
plt.legend(loc="lower right")
plt.xlabel('Recall (Sensitivity)', fontsize = 15)
plt.ylabel('Precision', fontsize = 15)

In [0]:
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['Cats', 'Dogs', 'Horse']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))